In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import ast

Data  Preprocessing

In [2]:
df_rating = pd.read_csv('../Movie_recommendations/Data/ratings.csv')
df_rating1 = df_rating.sample(n=3000, replace=False, random_state=1)
df_mv = pd.read_csv('../Movie_recommendations/Data/movies_metadata.csv')

/tmp/ipykernel_123280/900588374.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mv = pd.read_csv('../Movie_recommendations/Data/movies_metadata.csv')


In [3]:
# generate mv features
df_mv['id'] = df_mv['id'].astype(str)
df_rating['movieId'] = df_rating['movieId'].astype(str)
df_rate_mv_pro = pd.merge(df_mv, df_rating, how='right', left_on=['id'], right_on=['movieId'])


In [4]:
def get_genre(outterlist):
    outterlist = ast.literal_eval(outterlist)
    lst = []
    for a in outterlist:
        try:
            lst.append(a['name'])
        except IndexError:
            lst.append('')
    return '|'.join(lst)

In [5]:
df_rate_mv_pro1 = df_rate_mv_pro[['genres','id','title','userId','movieId', 'rating']]
df_rate_mv_pro1.dropna(inplace=True)
df_rate_mv_pro1['genres_descp'] = df_rate_mv_pro1['genres'].apply(get_genre)

/tmp/ipykernel_123280/1785784440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_mv_pro1.dropna(inplace=True)
/tmp/ipykernel_123280/1785784440.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_mv_pro1['genres_descp'] = df_rate_mv_pro1['genres'].apply(get_genre)


In [6]:
df_mv3 = df_rate_mv_pro1.loc[~df_rate_mv_pro1["id"].str.contains('-')]
df_mv3.drop_duplicates(subset =['id', 'title'], 
                     keep = 'first', inplace = True)
df_mv3 = df_mv3.loc[df_mv3['genres_descp'].str.len()>3]

In [7]:
df_mv4 = df_mv3.sample(n=3000, replace=False, random_state=1)
df_mv4.reset_index(inplace=True)

In [8]:
df_user_le = pd.DataFrame(df_mv4["userId"].tolist(), columns=['user'])
le = preprocessing.LabelEncoder()

df_user_le['user_le'] = le.fit_transform(df_user_le['user'])
df_mv_le = pd.DataFrame(df_mv4["movieId"].tolist(), columns=['movie'])
df_mv_le['mv_le'] = le.fit_transform(df_mv_le['movie'])

In [9]:
user_ids = np.array(df_user_le['user_le'].tolist())
movie_ids = np.array(df_mv_le['mv_le'].tolist())
user_ratings = np.array(df_mv4["rating"].tolist())
print ('num of users:',df_mv4['userId'].nunique())
print ('num of movies:',df_mv4['movieId'].nunique())

df = pd.DataFrame({'user': user_ids, 'movie': movie_ids, 'rating': user_ratings})

df.to_csv('Processed_Data.csv', index=False)

num of users: 1081
num of movies: 3000


In [11]:
# movieId Vs movie name
df_mv5 = df_mv4[['movieId', 'title']]
df_mv5.drop_duplicates(subset =['movieId', 'title'],
                        keep = 'first', inplace = True) 
df_mv5.to_csv('movieId_Vs_movie_name.csv', index=False)
# df_mv5

/tmp/ipykernel_123280/2780577925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mv5.drop_duplicates(subset =['movieId', 'title'],
